In [1]:
# Initial imports
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from pycoingecko import CoinGeckoAPI
import datetime as dt
from pytrends.request import TrendReq


%matplotlib inline

In [2]:
# Create a CoinGeckoAPI object
cg = CoinGeckoAPI()

# Get the current price of Bitcoin in USD
bitcoin_price = cg.get_price(ids='bitcoin', vs_currencies='usd')
print(bitcoin_price)


{'bitcoin': {'usd': 22704}}


In [3]:
# Define the cryptocurrencies and time range for the data
cryptos = ['bitcoin', 'ethereum', 'ripple']
vs_currency = 'usd'
days = 7

# Get the market data for each cryptocurrency
end_date = dt.datetime.now().date()
start_date = end_date - dt.timedelta(days=days)
market_info = []
for crypto in cryptos:
    data = cg.get_coin_market_chart_by_id(id=crypto, vs_currency=vs_currency, days=days)
    daily_data = {}
    for d in range(len(data['prices'])):
        timestamp = data['prices'][d][0] / 1000
        date = dt.datetime.fromtimestamp(timestamp).date()
        if start_date <= date <= end_date:
            midpoint_timestamp = dt.datetime(date.year, date.month, date.day, 12, 0, 0).timestamp()
            if timestamp > midpoint_timestamp:
                continue
            close = data['prices'][d][1]
            volume = data['total_volumes'][d][1]
            market_cap = data['market_caps'][d][1]
            if date not in daily_data:
                daily_data[date] = {'price': close, 'volume': volume, 'market_cap': market_cap, 'timestamp': timestamp}
            else:
                if abs(timestamp - midpoint_timestamp) < abs(daily_data[date]['timestamp'] - midpoint_timestamp):
                    daily_data[date] = {'price': close, 'volume': volume, 'market_cap': market_cap, 'timestamp': timestamp}
    for date, data in daily_data.items():
        ticker_name = crypto.upper()
        price = data['price']
        volume = data['volume']
        market_cap = data['market_cap']
        market_info.append([dt.datetime.fromtimestamp(data['timestamp']).strftime('%Y-%m-%d'), ticker_name, price, volume, market_cap])

# Print the market data
header = ['Date', 'Ticker Name', 'Price', 'Volume', 'Market Cap']
print(f'Daily market data for {", ".join(cryptos)} over the past {days} days:')
print(header)
for row in market_info:
    print(row)

Daily market data for bitcoin, ethereum, ripple over the past 7 days:
['Date', 'Ticker Name', 'Price', 'Volume', 'Market Cap']
['2023-02-08', 'BITCOIN', 22871.10332505254, 39728408728.34271, 440189053413.69257]
['2023-02-09', 'BITCOIN', 22553.628480160485, 42383060701.542404, 435099782557.33417]
['2023-02-10', 'BITCOIN', 21689.715252271788, 50704869298.319496, 418363793420.28595]
['2023-02-11', 'BITCOIN', 21713.467386147055, 27889757604.967937, 418531218933.1114]
['2023-02-12', 'BITCOIN', 22077.091346794103, 24041787020.215237, 425841151497.3257]
['2023-02-13', 'BITCOIN', 21504.98973004358, 35457775771.7496, 415163604741.9434]
['2023-02-14', 'BITCOIN', 22014.04173696712, 43011651007.45008, 423493624445.1542]
['2023-02-15', 'BITCOIN', 22735.46459338751, 45719910741.26019, 438762223789.2912]
['2023-02-08', 'ETHEREUM', 1645.6974444901214, 10559280030.506332, 197983515354.8018]
['2023-02-09', 'ETHEREUM', 1623.9144076632087, 9213619034.093061, 195736075841.03406]
['2023-02-10', 'ETHEREUM', 

In [9]:
# Get social media data for Bitcoin
bitcoin_data = cg.get_coin_by_id(id='bitcoin')

print(bitcoin_data['community_data'])

{'facebook_likes': None, 'twitter_followers': 5719372, 'reddit_average_posts_48h': 6.417, 'reddit_average_comments_48h': 217.833, 'reddit_subscribers': 4824257, 'reddit_accounts_active_48h': 8271, 'telegram_channel_user_count': None}


In [10]:
# Print the social media data
print(f"Bitcoin's social media data:")
print(f"Twitter followers: {bitcoin_data['community_data']['twitter_followers']}")
print(f"Reddit subscribers: {bitcoin_data['community_data']['reddit_subscribers']}")



Bitcoin's social media data:
Twitter followers: 5719372
Reddit subscribers: 4824257


In [6]:
# Import required libraries for news API Request
from __future__ import print_function
import time
import aylien_news_api
from aylien_news_api.rest import ApiException
from pprint import pprint
import os
from dotenv import load_dotenv

# Check .env is being called
load_dotenv()




True

In [7]:
# Set API Key and ID Variables and Reference to .env file

api_id = os.getenv('X-AYLIEN-NewsAPI-Application-ID')
api_key = os.getenv('X-AYLIEN-NewsAPI-Application-Key')

configuration = aylien_news_api.Configuration()
configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = api_id
configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = api_key

# Set Client variable
client = aylien_news_api.ApiClient(configuration)
api_instance = aylien_news_api.DefaultApi(client)

# Request news stories related to title 'keyword'
try:
    api_response = api_instance.list_stories(
        title='bitcoin',
        published_at_start='NOW-7DAYS',
        published_at_end='NOW'
    )
    pprint(api_response)
except ApiException as e:
    print("Exception when calling DefaultApi->list_stories: %s\n" % e)

{'next_page_cursor': 'MTY3NjQ3MDIzNjAwMCw1MzA1OTQ1ODEz',
 'published_at_end': datetime.datetime(2023, 2, 15, 14, 21, 18, 824443, tzinfo=tzutc()),
 'published_at_start': datetime.datetime(2023, 2, 8, 14, 21, 18, 824411, tzinfo=tzutc()),
 'stories': [{'author': {'avatar_url': None, 'id': 1000068256, 'name': ''},
              'body': 'Ordinals keep making headlines as there are now more '
                      'than 100,000 inscriptions carried out on Bitcoin’s '
                      'network. This has had a notable effect on the network '
                      'state itself and has sparked a massive debate within '
                      'the community.\n'
                      '\n'
                      'Inscriptions are Bitcoin’s version of a non-fungible '
                      'token (NFT), and they have been making strides '
                      'throughout the past couple of weeks.\n'
                      '\n'
                      'Multiple…\n'
                      '\n'
      